In [ ]:
class LL1Parser:
    def __init__(self, grammar):
        self.grammar = grammar
        self.non_terminals = set()
        self.terminals = set()
        self.first_sets = {}
        self.follow_sets = {}
        self.parsing_table = {}

    def compute_first_sets(self):
        # Initialize first sets
        for non_terminal in self.grammar.keys():
            self.first_sets[non_terminal] = set()

        # Calculate first sets iteratively
        while True:
            prev_first_sets = {key: value.copy() for key, value in self.first_sets.items()}

            for non_terminal, productions in self.grammar.items():
                for production in productions:
                    first_symbol = production[0]

                    if first_symbol in self.terminals:
                        self.first_sets[non_terminal].add(first_symbol)
                    elif first_symbol in self.non_terminals:
                        self.first_sets[non_terminal].update(self.first_sets[first_symbol])

            if prev_first_sets == self.first_sets:
                break

    def compute_follow_sets(self, start_symbol):
        # Initialize follow sets
        for non_terminal in self.grammar.keys():
            self.follow_sets[non_terminal] = set()

        # Add $ (end of input) to the follow set of the start symbol
        self.follow_sets[start_symbol].add('$')

        # Calculate follow sets iteratively
        while True:
            prev_follow_sets = {key: value.copy() for key, value in self.follow_sets.items()}

            for non_terminal, productions in self.grammar.items():
                for production in productions:
                    for i, symbol in enumerate(production):
                        if symbol in self.non_terminals:
                            if i < len(production) - 1:
                                next_symbol = production[i + 1]
                                if next_symbol in self.terminals:
                                    self.follow_sets[symbol].add(next_symbol)
                                else:
                                    self.follow_sets[symbol].update(self.first_sets[next_symbol])
                            else:
                                self.follow_sets[symbol].update(self.follow_sets[non_terminal])

            if prev_follow_sets == self.follow_sets:
                break

    def construct_parsing_table(self):
        for non_terminal, productions in self.grammar.items():
            for production in productions:
                first_set = self.compute_production_first_set(production)
                for terminal in first_set:
                    if terminal != 'ε':
                        self.parsing_table[(non_terminal, terminal)] = production
                if 'ε' in first_set:
                    follow_set = self.follow_sets[non_terminal]
                    for terminal in follow_set:
                        self.parsing_table[(non_terminal, terminal)] = production

    def compute_production_first_set(self, production):
        first_set = set()
        for symbol in production:
            if symbol in self.terminals:
                first_set.add(symbol)
                break
            elif symbol in self.non_terminals:
                first_set.update(self.first_sets[symbol])
                if 'ε' not in self.first_sets[symbol]:
                    break
        return first_set

    def print_parsing_table(self):
        for key, value in self.parsing_table.items():
            print(f"M[{key[0]}, {key[1]}] = {value}")

    def construct_ll1_parsing_table(self, start_symbol):
        self.non_terminals = set(self.grammar.keys())
        self.terminals = set(
            symbol
            for productions in self.grammar.values()
            for production in productions
            for symbol in production
            if symbol not in self.non_terminals
        )

        self.compute_first_sets()
        self.compute_follow_sets(start_symbol)
        self.construct_parsing_table()


if __name__ == "__main__":
    # Defined CFG:
    grammar = {
       
    'E' : ['E + T' ,'E - T' , 'T'],
    'T' : ['T * F', 'T / F', 'F'],
    'F' : ['F ^ G', 'G']
    }
    


    start_symbol = 'E'

    parser = LL1Parser(grammar)
    parser.construct_ll1_parsing_table(start_symbol)
    parser.print_parsing_table()
